# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
sns.set_palette('muted')

import plotly.graph_objects as go

In [2]:
DATA_RAW_PATH = '../../data/raw/'

In [3]:
btc_df = pd.read_csv(DATA_RAW_PATH + 'historic_data.csv')

# Plots

In [4]:
btc_df.head()

,time,open,high,low,close,vwap,volume,count
0,2024-10-11 08:30:00,60703.4,60703.4,60666.1,60666.1,60685.4,0.917922,80
1,2024-10-11 08:35:00,60660.0,60668.1,60641.9,60641.9,60665.1,0.951402,81
2,2024-10-11 08:40:00,60641.9,60642.0,60640.0,60640.1,60640.0,2.144969,143
3,2024-10-11 08:45:00,60640.1,60640.1,60635.0,60640.0,60635.4,2.928036,30
4,2024-10-11 08:50:00,60635.6,60640.1,60621.1,60640.1,60638.0,1.687531,89


In [5]:
btc_df['moving_average_10'] = btc_df['close'].rolling(window=10).mean()

In [6]:
btc_df['moving_average'] = btc_df['close'].rolling(window=20).mean()
btc_df['std_deviation'] = btc_df['close'].rolling(window=20).std()
btc_df['upper_band'] = btc_df['moving_average'] + (btc_df['std_deviation'] * 2)
btc_df['lower_band'] = btc_df['moving_average'] - (btc_df['std_deviation'] * 2)

In [39]:
sub_btc_df = btc_df.iloc[300:400]

fig = go.Figure(data=[go.Candlestick(
    x=sub_btc_df['time'],
    open=sub_btc_df['open'],
    high=sub_btc_df['high'],
    low=sub_btc_df['low'],
    close=sub_btc_df['close']
)])

# Moving Average
fig.add_trace(go.Scatter(
    x=sub_btc_df['time'],
    y=sub_btc_df['moving_average'],
    mode='lines',
    line=dict(color='orange', width=2),
    name='20-period Moving Average'
))

# Upper Band
fig.add_trace(go.Scatter(
    x=sub_btc_df['time'],
    y=sub_btc_df['upper_band'],
    mode='lines',
    line=dict(color='green', width=1, dash='dash'),
    name='Upper Bollinger Band'
))

# Lower Band
fig.add_trace(go.Scatter(
    x=sub_btc_df['time'],
    y=sub_btc_df['lower_band'],
    mode='lines',
    line=dict(color='red', width=1, dash='dash'),
    name='Lower Bollinger Band'
))

fig.update_layout(
    width=1200,
    height=600,
    xaxis_title='Date',
    yaxis_title='Price',
    plot_bgcolor='rgb(40, 40, 40, 100)',
    paper_bgcolor='rgb(40, 40, 40, 100)',
    font=dict(color='rgb(200, 200, 200, 100)'),
    xaxis=dict(
        showline=True,
        linecolor='rgb(200, 200, 200, 100)',
        linewidth=2,
        gridcolor='rgb(200, 200, 200, 100)',
        zerolinecolor='rgb(200, 200, 200, 100)'
    ),
    yaxis=dict(
        showline=True,
        linecolor='rgb(200, 200, 200, 100)',
        linewidth=2,
        gridcolor='rgb(200, 200, 200, 100)',
        zerolinecolor='rgb(200, 200, 200, 100)'
    )
)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [51]:
from statsmodels.tsa.stattools import adfuller

close_prices = btc_df['close']

adf_test = adfuller(close_prices)

print('ADF Statistic:', adf_test[0])
print('p-value:', adf_test[1])
print('Critical Values:')

for key, value in adf_test[4].items():
    print(f'\t{key}: {value}')

if adf_test[1] > 0.05:
    print('Time Series is not Stationary')
else:
    print('Time Series is Stationary')

ADF Statistic: -3.544503782923574
p-value: 0.006915054788555855
Critical Values:
	1%: -3.439477677285233
	5%: -2.865568186630048
	10%: -2.5689150720653977
Time Series is Stationary


In [49]:
# Calcular los rendimientos logarítmicos
btc_returns = np.log(btc_df['close']).diff().dropna()

# Realizar la prueba ADF en los rendimientos
adf_test = adfuller(btc_returns)

print("ADF Statistic:", adf_test[0])
print("p-value:", adf_test[1])
if adf_test[1] > 0.05:
    print("La serie de rendimientos tiene una raíz unitaria y probablemente sigue un camino aleatorio.")
else:
    print("La serie de rendimientos es estacionaria.")

ADF Statistic: -16.40174374945744
p-value: 2.6341748593003395e-29
La serie de rendimientos es estacionaria.
